In [ ]:
# run this for working with colab
import torch.cuda as cuda
cuda.get_device_name()

from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/lab/SSD/"
dir = "/content/gdrive/My Drive/lab/SSD/"


In [1]:
import os
import torch
import torch.nn as nn
import torch.cuda as cuda
from torch.utils.data import DataLoader, random_split
from dataset import VOCDataset, collate_fn
#from dataset_h5 import VOCDataset
from model import SSD300, MultiBoxLoss
from utils import *

device = torch.device("cuda:0" if cuda.is_available() else "cpu") 

## Load dataset

In [ ]:
#from utils import create_data_lists
voc07_path = 'D:/C-dir/SSD/VOCdevkit/VOC2007/'
voc12_path = 'D:/C-dir/SSD/VOCdevkit/VOC2012/'
create_data_lists(voc07_path, voc12_path, 'D:/C-dir/SSD/VOCdevkit/')

In [2]:
torch.manual_seed(42)
trainset = VOCDataset(data_folder='datasets/', is_trainset=True)
train_size = int(0.9 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = random_split(trainset, [train_size, val_size])
testset = VOCDataset(data_folder='datasets/', is_trainset=False)

dataloaders = dict(
    train = DataLoader(trainset, batch_size=8, collate_fn=collate_fn, shuffle=True, num_workers=2),
    val = DataLoader(valset, batch_size=64, collate_fn=collate_fn, shuffle=False, num_workers=2),
    test = DataLoader(testset, batch_size=64, collate_fn=collate_fn, shuffle=False, num_workers=2)
)

## Train model

In [3]:
MySSD300 = SSD300(n_classes = 21)

In [4]:
checkpoint = None
loss_func = MultiBoxLoss(priors_cxcy = MySSD300.get_prior_boxes(), threshold=0.5, neg_pos_ratio=3, alpha=1.)
optimizer = torch.optim.Adam(MySSD300.parameters(), lr=1e-3, weight_decay=5e-4)
grad_clip = None


In [5]:
def train_epoch(model, dataloaders, loss_func, optimizer):
    model.train()
    epoch_loss = 0
    for step, (imgs, boxes, labels) in enumerate(dataloaders['train']):
        offsets, class_scores = model(imgs)
        loss = loss_func(offsets, class_scores, boxes, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print('Step:', step, '- Done - loss:', loss.item())
    return epoch_loss/len(dataloaders['train'])

def eval(model, dataloaders):
    model.eval()
    
    # torch no_grad()
    
    epoch_loss = 0
    for step, (imgs, boxes, labels) in enumerate(dataloaders['val']):
        offsets, class_scores = model(imgs)
        loss = loss_func(pred, y)
        epoch_loss += loss.item()
        
        #calculate accuracy
        det_boxes_batch, det_labels_batch, det_scores_batch = model.detect_objects(predicted_locs, predicted_scores,
                                                                                    min_score=0.01, max_overlap=0.45, top_k=200)
        
        # Calculate mAP
        APs, mAP = calculate_mAP(det_boxes, det_labels, det_scores, true_boxes, true_labels, true_difficulties)
        
    loss = total_loss/len(dataloaders['val'])
    
    
    return loss, accuracy


def train_model(model, dataloaders, epoch=1, ):
    
    #if checkpoint is None:
    
    for i in range(epoch):
        train_epoch(model, dataloaders)
    
    save_checkpoint(epoch, model, optimizer)

In [6]:
#Try
train_epoch(MySSD300, dataloaders)

Step: 0 - Done - loss: 84032.7265625
Step: 1 - Done - loss: 21933.30078125
Step: 2 - Done - loss: 11335.4013671875
Step: 3 - Done - loss: 5332.73388671875
Step: 4 - Done - loss: 9620.279296875
Step: 5 - Done - loss: 3608.971923828125
Step: 6 - Done - loss: 2503.865966796875
Step: 7 - Done - loss: 2496.44384765625


KeyboardInterrupt: 